In [2]:
import boto3
from boto.s3.connection import S3Connection
import os
import json
import boto.s3
import sys
import datetime
from boto.s3.key import Key
from pprint import pprint
import pandas as pd
import urllib
import csv
import io
import requests
import time
import json
import datetime
from pprint import pprint
import scipy
import numpy as np

#Loding data from csv 

rawdata= pd.read_csv("properties_2016.csv")
        
print(rawdata.head(5))



        
        

C:\Users\sneha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   parcelid  airconditioningtypeid  architecturalstyletypeid  basementsqft  \
0  10754147                    NaN                       NaN           NaN   
1  10759547                    NaN                       NaN           NaN   
2  10843547                    NaN                       NaN           NaN   
3  10859147                    NaN                       NaN           NaN   
4  10879947                    NaN                       NaN           NaN   

   bathroomcnt  bedroomcnt  buildingclasstypeid  buildingqualitytypeid  \
0          0.0         0.0                  NaN                    NaN   
1          0.0         0.0                  NaN                    NaN   
2          0.0         0.0                  NaN                    NaN   
3          0.0         0.0                  3.0                    7.0   
4          0.0         0.0                  4.0                    NaN   

   calculatedbathnbr  decktypeid         ...           numberofstories  \
0           

In [3]:
rawdata=rawdata[["parcelid","airconditioningtypeid","architecturalstyletypeid","basementsqft",
                 "bedroomcnt","buildingclasstypeid","buildingqualitytypeid","decktypeid",
                 "finishedfloor1squarefeet","calculatedfinishedsquarefeet","finishedsquarefeet12",
                 "finishedsquarefeet13","finishedsquarefeet15","finishedsquarefeet50","finishedsquarefeet6",
                 "fips","fireplacecnt","fullbathcnt","garagecarcnt","garagetotalsqft","hashottuborspa",
                 "heatingorsystemtypeid","latitude","longitude","lotsizesquarefeet","poolcnt",
                 "propertycountylandusecode","propertylandusetypeid",
                 "propertyzoningdesc","rawcensustractandblock","regionidcity","regionidcounty",
                 "regionidzip","roomcnt","threequarterbathnbr",
                 "yardbuildingsqft17","yardbuildingsqft26","yearbuilt",
                 "numberofstories","fireplaceflag","structuretaxvaluedollarcnt",
                 "assessmentyear","landtaxvaluedollarcnt","taxamount","taxdelinquencyyear",
                 "censustractandblock"]]

rawdata.head(5)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,...,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015.0,9.0,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015.0,27516.0,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,73026.0,...,NaN,NaN,NaN,NaN,650756.0,2015.0,762631.0,20800.37,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,3.0,7.0,NaN,NaN,5068.0,...,NaN,1948.0,1.0,NaN,571346.0,2015.0,585488.0,14557.57,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,4.0,NaN,NaN,NaN,1776.0,...,NaN,1947.0,NaN,NaN,193796.0,2015.0,239695.0,5725.17,NaN,NaN


In [8]:
print(rawdata.shape)

rawdata.describe()


(2985217, 49)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,...,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,2.985217e+06,811519.000000,6061.000000,1628.000000,2.973767e+06,12629.000000,1.938488e+06,17096.0,202717.000000,2.929652e+06,...,80355.000000,2647.000000,2.925289e+06,682069.000000,2.930235e+06,2.973778e+06,2.917484e+06,2.953967e+06,56464.000000,2.910091e+06
mean,1.332586e+07,1.931166,7.202607,646.883292,3.088949e+00,3.725948,5.784787e+00,66.0,1380.630396,1.827162e+03,...,319.803397,278.296562,1.964262e+03,1.401464,1.708836e+05,2.014999e+03,2.524780e+05,5.377607e+03,13.892409,6.048431e+13
std,7.909966e+06,3.148587,2.436290,538.793473,1.275859e+00,0.501700,1.805352e+00,0.0,632.868543,1.819780e+03,...,233.086314,369.731508,2.344132e+01,0.539076,4.020683e+05,3.683161e-02,4.450132e+05,9.183107e+03,2.581006,3.249035e+11
min,1.071172e+07,1.000000,2.000000,20.000000,0.000000e+00,1.000000,1.000000e+00,66.0,3.000000,1.000000e+00,...,10.000000,10.000000,1.801000e+03,1.000000,1.000000e+00,2.000000e+03,1.000000e+00,1.340000e+00,0.000000,-1.000000e+00
25%,1.164371e+07,1.000000,7.000000,272.000000,2.000000e+00,3.000000,4.000000e+00,66.0,1012.000000,1.213000e+03,...,190.000000,96.000000,1.950000e+03,1.000000,7.480000e+04,2.015000e+03,7.483600e+04,2.461070e+03,14.000000,6.037400e+13
50%,1.254509e+07,1.000000,7.000000,534.000000,3.000000e+00,4.000000,7.000000e+00,66.0,1283.000000,1.572000e+03,...,270.000000,168.000000,1.963000e+03,1.000000,1.225900e+05,2.015000e+03,1.670420e+05,3.991780e+03,14.000000,6.037572e+13
75%,1.409712e+07,1.000000,7.000000,847.250000,4.000000e+00,4.000000,7.000000e+00,66.0,1615.000000,2.136000e+03,...,390.000000,320.000000,1.981000e+03,2.000000,1.968890e+05,2.015000e+03,3.069180e+05,6.201005e+03,15.000000,6.059042e+13
max,1.696019e+08,13.000000,27.000000,8516.000000,2.000000e+01,5.000000,1.200000e+01,66.0,31303.000000,9.525760e+05,...,7983.000000,6141.000000,2.015000e+03,41.000000,2.514860e+08,2.016000e+03,9.024622e+07,3.458861e+06,99.000000,4.830301e+14


In [9]:
rawdata.dtypes


parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                   object
heatingorsystemtypeid           float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64


In [20]:
# df1=rawdata[["poolcnt",
#                  "pooltypeid10"]]

# df1.fillna(0, inplace=True)

# df1.head(5)

# df1.describe()


df1=rawdata[["airconditioningtypeid","architecturalstyletypeid","basementsqft",
                 "bedroomcnt","buildingclasstypeid","buildingqualitytypeid","decktypeid",
                 "finishedfloor1squarefeet","calculatedfinishedsquarefeet","finishedsquarefeet12",
                 "finishedsquarefeet13","finishedsquarefeet15","finishedsquarefeet50","finishedsquarefeet6",
                 "fips","fireplacecnt","fullbathcnt","garagecarcnt","garagetotalsqft","hashottuborspa",
                 "heatingorsystemtypeid","latitude","longitude","lotsizesquarefeet","poolcnt",
                 "propertycountylandusecode","propertylandusetypeid",
                 "propertyzoningdesc","rawcensustractandblock","regionidcity","regionidcounty",
                 "regionidzip","roomcnt","threequarterbathnbr",
                 "yardbuildingsqft17","yardbuildingsqft26","yearbuilt",
                 "numberofstories","fireplaceflag","structuretaxvaluedollarcnt",
                 "assessmentyear","landtaxvaluedollarcnt","taxamount","taxdelinquencyyear",
                 "censustractandblock"]]




C:\Users\sneha\Anaconda3\lib\site-packages\pandas\core\frame.py:2842: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,poolcnt,pooltypeid10
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [21]:
df1.describe()

,poolcnt,pooltypeid10
count,2.985217e+06,2.985217e+06
mean,1.733656e-01,1.237397e-02
std,3.785631e-01,1.105480e-01
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00


In [18]:
# df1["poolcnt"]= df1["poolcnt"]+ df1["pooltypeid10"]
# df1.describe()




C:\Users\sneha\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,poolcnt,pooltypeid10
count,2.985217e+06,2.985217e+06
mean,1.857396e-01,1.237397e-02
std,3.888965e-01,1.105480e-01
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00


In [25]:

rawdata["parcelid"].shape
rawdata["parcelid"].tail(10)



# df1[['col2','col3']] = df[['col2','col3']].apply(pd.to_numeric)

2985207     14460030
2985208     14284830
2985209     14285230
2985210     14455630
2985211     11117630
2985212    168176230
2985213     14273630
2985214    168040630
2985215    168040830
2985216    168040430
Name: parcelid, dtype: int64